# FLIP (04): Learning Theory (I)
**(Module 03: Operation Research)**

---
- Materials in this module include resources collected from various open-source online repositories.
- You are free to use, but NOT allowed to change or distribute this package.

Prepared by and for 
**Student Members** |
2006-2018 [TULIP Lab](http://www.tulip.org.au), Australia

---

# Blending and Mixture Problems

In [ ]:
import pulp

def dietlp(price,nutrition,requirements, nmin = 0, nmax=100):
    diet = pulp.LpProblem('Diet Problem', pulp.LpMinimize)
    x = pulp.LpVariable.dicts("x", list(price.index), nmin, nmax, pulp.LpInteger)

    diet += pulp.lpSum([price.ix[idx,'price']*x[idx] for idx in list(price.index)])

    for n in nutrition.columns:
        diet += pulp.lpSum([nutrition.ix[idx,n]*x[idx] for idx in list(price.index)]) >= requirements.ix[n,'n_min'], n + '_min'
        diet += pulp.lpSum([nutrition.ix[idx,n]*x[idx] for idx in list(price.index)]) <= requirements.ix[n,'n_max'], n + '_max'
    
    return diet

diet = dietlp(price,nutrition,requirements,0,10)


## Solution

In [ ]:
def solvelp(lp):
    
    lp.solve();

    obj = pd.DataFrame(
        [[lp.objective.name, 
          pulp.LpSenses[lp.sense], 
          pulp.LpStatus[lp.status], 
          pulp.value(lp.objective)]],
        columns=["Objective", "Sense", "Status", "Value"])

    var = pd.DataFrame(
        [[x.name, x.cat, x.lowBound, x.upBound, x.varValue] for x in lp.variables()],
        columns=["Name","Category","Lower Bound","Upper Bound","Value"])    

    con = pd.DataFrame(
        [[c.name, 
          pulp.LpConstraintSenses[c.sense], 
          c.getLb(), 
          c.getUb(), 
          c.slack, 
          pulp.value(c) - c.constant] for (k,c) in lp.constraints.items()],
        columns=["Name", 'Sense', "Lower Bound", "Upper Bound", "Slack", "Value"])
    
    return {'Objective':obj, 'Variables':var, 'Constraints':con}

soln = solvelp(diet)

## Display

In [ ]:
from IPython.display import display,HTML

display(HTML('<h3>Model</h3>'))
display(diet)
for k in soln.keys():
    display(HTML('<h3>' + k + '</h3>'))
    display(soln[k])
    

In [ ]:
ax = soln['Variables'].ix[:,'Value'].plot(kind='bar')
ax.set_xticklabels(list(soln['Variables'].ix[:,'Name']));

In [ ]:
nmax = arange(0,60,1)
obj = zeros(size(nmax))
for n in nmax:
    soln = solvelp(dietlp(price,nutrition,requirements,0,n));
    obj[n-1] = soln['Objective'].ix[0,'Value'] if (soln['Objective'].ix[0,'Status'] == 'Optimal') else NaN

plot(nmax,obj,'r.')
xlabel('n_max')
ylabel('Cost')

## Refinery Operations

In [ ]:
%%script glpsol -m /dev/stdin -o /dev/stdout --out output

set COMPONENTS;
set PRODUCTS;
set CRUDES;

param feed {COMPONENTS,CRUDES};
param avail {CRUDES}; 

data;

set COMPONENTS := 
    Butane  
    Naptha  
    Gasoil  
    Residue ;
    
set CRUDES :=
    Crude_1
    Crude_2 ;
    
param feed :  Crude_1   Crude_2 :=
    Butane     0.03      0.05
    Naphtha    0.15      0.20
    Gasoil     0.40      0.35
    Residue    0.15      0.10 ;
    
param avail :=
    Crude_1    250000
    Crude_2    400000 ;
    
end;



In [ ]:
print output